In [45]:
import scipy.stats as st
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from statsmodels.stats.proportion import proportions_ztest
from statsmodels.stats.multitest import multipletests

In [46]:
data = pd.read_csv("data/nba.csv", index_col=0)
data.head()

,player,team,atm_away,atm_home,score_away,score_home,atm_total,score_total
7,A. Johnson,ATL,30,34,24,28,64,52
10,A. Johnson,TOR,63,57,44,38,120,82
15,Abdur-Rahim,SAC,114,124,84,90,238,174
22,Afflalo,DEN,236,284,189,227,520,416
23,Afflalo,DET,75,88,62,67,163,129


Проверим гипотезу о равенстве долей успешных бросков в домашних и гостевых играх с помощью Z-критрия для долей (независимые выборки).

In [90]:
p_vals = []
for i in range(len(data)):
    entry = data.iloc[i, :]
    succ = entry["score_away"], entry["score_home"]
    trial = entry["atm_away"], entry["atm_home"]
    p_vals.append(proportions_ztest(succ, trial, alternative="two-sided")[1])
p_vals = np.array(p_vals)

In [67]:
(p_vals < 0.05).sum()

65

Без коррекции на множественную проверку гипотез мы бы отвергли 65 гипотез, однако это было бы некорректно. Используем метод Холма для учета множественной проверки.

In [63]:
multipletests(p_vals, method='holm')[0].sum()

0

Метод Холма не позволяет отвергнуть ни одной гипотезы.

Возможно, что нам не хватает объема выборки для отвержения гипотез. Попробуем объеденить выборки для одних и тех же игроков из разных команд.

In [76]:
players = list(set(data['player']))

In [95]:
p_vals = []
for name in players:
    entry = data[data['player'] == name].sum(axis=0)
    succ = entry["score_away"], entry["score_home"]
    trial = entry["atm_away"], entry["atm_home"]
    p_vals.append(proportions_ztest(succ, trial, alternative="two-sided")[1])
p_vals = np.array(p_vals)

In [96]:
multipletests(p_vals, method='holm')[0].sum()

1

In [97]:
multipletests(p_vals, method='fdr_bh')[0].sum()

1

In [87]:
players[multipletests(p_vals, method='holm')[0].argmax()]

'Martin'

**Вывод**

Таким образом Z-критерий для доли позволил отвергнуть гипотезу о независимости успешности бросков от локации игры лишь для одного игрока (Martin) из 458. Причем этот результат довольно устойчив к методике коррекции на множественную проверку, так как даже при переходе от котроля FWER (Холм) к FDR (Бенджамини-Хохберг) результат не изменился.